In [2]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="postgres",
    password="123456",
    port="5432"
)

cursor = conn.cursor()
print("Connected to PostgreSQL")




Connected to PostgreSQL


In [3]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS banks (
    bank_id SERIAL PRIMARY KEY,
    bank_code TEXT UNIQUE,
    bank_name TEXT NOT NULL,
    app_id TEXT,
    current_rating REAL,
    total_ratings INTEGER,
    total_reviews INTEGER
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS reviews (
    review_id TEXT PRIMARY KEY,
    bank_id INTEGER REFERENCES banks(bank_id) ON DELETE CASCADE,
    review_text TEXT,
    rating SMALLINT,
    review_date DATE,
    review_year SMALLINT,
    review_month SMALLINT,
    user_name TEXT,
    thumbs_up INTEGER,
    text_length INTEGER,
    source TEXT,
    sentiment_label TEXT,
    sentiment_score REAL
);
""")

conn.commit()
print("Tables 'banks' and 'reviews' created successfully.")


Tables 'banks' and 'reviews' created successfully.


In [15]:
import pandas as pd

df = pd.read_csv("../notebooks/data/processed/reviews_processed.csv")
df.head()

,review_id,review_text,rating,review_date,review_year,review_month,bank_code,bank_name,user_name,thumbs_up,text_length,source
0,fc67d12c-92e2-45aa-a9e0-011f58a583bc,goof,5,2025-11-28,2025,11,Abyssinia,Abyssinia Bank,Hailegebrail Tegegn,0,4,Google Play
1,11306fb9-5571-4950-8d32-604c5402242f,good!,5,2025-11-28,2025,11,Abyssinia,Abyssinia Bank,Tsegay ab,0,5,Google Play
2,809c46d2-730e-446a-9061-2a45e978ad9d,good jop,5,2025-11-27,2025,11,Abyssinia,Abyssinia Bank,Yohanis Fikadu,0,8,Google Play
3,f28a3a3c-eb94-4aab-88d2-89bcecebcc7b,bad exprience...it is so crushed,1,2025-11-27,2025,11,Abyssinia,Abyssinia Bank,ሻንበል ኪዳነ,0,32,Google Play
4,4ed89e8c-16dc-4763-94ca-04d05cf799a5,not user friendly at all it requires a huge co...,1,2025-11-26,2025,11,Abyssinia,Abyssinia Bank,Tsegaw Fikru,0,85,Google Play


In [5]:
conn.rollback()
print("Transaction reset. You can now run SQL commands again.")


Transaction reset. You can now run SQL commands again.


In [17]:
# Show column names with markers to detect spaces
for col in df.columns:
    print(repr(col))


'review_id'
'review_text'
'rating'
'review_date'
'review_year'
'review_month'
'bank_code'
'bank_name'
'user_name'
'thumbs_up'
'text_length'
'source'


In [19]:
from psycopg2.extras import execute_values
import pandas as pd

# Fetch bank_id mapping from the banks table
cursor.execute("SELECT bank_id, bank_code FROM banks;")
bank_map = {row[1]: row[0] for row in cursor.fetchall()}  # e.g., {'Abyssinia': 1, 'CBE': 2, ...}

# Add bank_id column to DataFrame
df['bank_id'] = df['bank_code'].map(bank_map)

# Optional: check for unmapped bank codes
unmapped = df[df['bank_id'].isna()]['bank_code'].unique()
if len(unmapped) > 0:
    print("Unmapped bank codes:", unmapped)


# 4. Convert DF rows → list of tuples (ensure proper Python types)
data = [
    (
        str(row.review_id),
        int(row.bank_id) if pd.notna(row.bank_id) else None,
        str(row.review_text),
        int(row.rating) if pd.notna(row.rating) else None,
        row.review_date,  # already datetime, OK for PostgreSQL
        int(row.review_year),
        int(row.review_month),
        str(row.user_name),
        int(row.thumbs_up) if pd.notna(row.thumbs_up) else None,
        int(row.text_length),
        str(row.source),
        None,
        None
    )
    for row in df.itertuples(index=False)
]

# 5. Correct INSERT query (must match your table schema)
insert_query = """
INSERT INTO reviews (
    review_id, bank_id, review_text, rating, review_date,
    review_year, review_month, user_name, thumbs_up,
    text_length, source, sentiment_label, sentiment_score
) VALUES %s
"""

# 6. Insert into PostgreSQL
execute_values(cursor, insert_query, data)
conn.commit()

print("Inserted", len(data), "rows successfully!")


Unmapped bank codes: ['Abyssinia' 'CBE' 'Dashen']
Inserted 938 rows successfully!


In [23]:
conn.rollback()
print("Transaction reset. You can now run SQL commands again.")


Transaction reset. You can now run SQL commands again.


In [ ]:
from psycopg2.extras import execute_values
import pandas as pd

# --- 1. Insert missing banks ---
banks = [
    ('Abyssinia', 'Abyssinia Bank', 'com.boa.boaMobileBanking', 4.196262, 6328, 1219),
    ('CBE', 'Commercial Bank of Ethiopia', 'com.combanketh.mobilebanking', 4.322109, 38852, 8335),
    ('Dashen', 'Dashen', 'com.dashen.dashensuperapp', 4.171717, 3901, 771)
]

insert_banks = """
INSERT INTO banks (bank_code, bank_name, app_id, current_rating, total_ratings, total_reviews)
VALUES %s
ON CONFLICT (bank_code) DO NOTHING;
"""

execute_values(cursor, insert_banks, banks)
conn.commit()
print("Banks inserted/updated successfully!")




Banks inserted/updated successfully!


In [25]:
# 1. Map bank_code → bank_id in the DataFrame
df['bank_id'] = df['bank_code'].map(bank_map)

# 2. Prepare only rows where bank_id is now set (ignore missing)
df_to_update = df[df['bank_id'].notna()]

# 3. Convert rows to list of tuples for insertion
from psycopg2.extras import execute_values

data = [
    (
        str(row.review_id),
        int(row.bank_id),
        str(row.review_text),
        int(row.rating),
        row.review_date,
        int(row.review_year),
        int(row.review_month),
        str(row.user_name),
        int(row.thumbs_up),
        int(row.text_length),
        str(row.source),
        None,   # sentiment_label placeholder
        None    # sentiment_score placeholder
    )
    for row in df_to_update.itertuples(index=False)
]

# 4. Insert into PostgreSQL
insert_query = """
INSERT INTO reviews (
    review_id, bank_id, review_text, rating, review_date,
    review_year, review_month, user_name, thumbs_up,
    text_length, source, sentiment_label, sentiment_score
) VALUES %s
ON CONFLICT (review_id) DO UPDATE SET
    bank_id = EXCLUDED.bank_id
"""

execute_values(cursor, insert_query, data)
conn.commit()

print("Inserted/updated", len(data), "reviews successfully!")


Inserted/updated 938 reviews successfully!


In [26]:
#  --- 2. Reload bank mapping ---
cursor.execute("SELECT bank_id, bank_code FROM banks;")
bank_map = {row[1]: row[0] for row in cursor.fetchall()}
# --- Optional: verify total reviews ---
cursor.execute("SELECT COUNT(*) FROM reviews;")
total_reviews = cursor.fetchone()[0]
print("Total reviews in DB:", total_reviews)

Total reviews in DB: 938
